# Mount drive and bucket
Todo: Remove in public version

In [1]:
# Check if the notebook is run in Google Colab
import sys

COLAB = 'google.colab' in sys.modules

In [2]:
import sys
import datetime
import logging

nblog = open(f"/content/bigdata/nb_{datetime.datetime.utcnow().strftime('%Y%m%d_%H%M')}.log", "a+")
print(nblog)
sys.stdout.echo = nblog
sys.stderr.echo = nblog

#get_ipython().log.handlers[0].stream = nblog
#get_ipython().log.setLevel(logging.INFO)

%autosave 60

<_io.TextIOWrapper name='/content/bigdata/nb_20230227_1520.log' mode='a+' encoding='UTF-8'>


Autosaving every 60 seconds


In [3]:
# if COLAB:
#   from google.colab import drive
#   drive.mount('/content/drive')

In [4]:
if COLAB:
  from google.colab import auth
  auth.authenticate_user()
else:
    print("Run the command!")
  #Todo #bring the command inside the notebook
  #run this terminal inside docker: gcloud auth login b.girsule@gmail.com --no-launch-browser

Run the command!


In [5]:
# Todo: Check if possible in local docker
# from google.cloud import storage

In [6]:
# Check if the tf version is 2.10.0, this is required to use the 'ignore_class' in the  SparseCategoricalCrossentropy
import tensorflow as tf

if '2.10.0' != tf.__version__:
  !pip uninstall tensorflow -y
  !pip install tensorflow-gpu==2.10.0
  please_restart_the_runtime
else:
  print("TF Version is 2.10.0, ok!")

2023-02-27 15:20:26.213679: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-27 15:20:26.337773: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-02-27 15:20:26.968837: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-27 15:20:26.968894: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

TF Version is 2.10.0, ok!


In [7]:
from tensorflow import keras
# import tensorflow_gcs_config
from tensorflow.python.lib.io import file_io

from keras.layers import Input, Dense, Flatten #, ReLU, Add, Flatten, Concatenate, LayerNormalization, UpSampling2D, Activation, LSTM, Multiply, Dropout, Reshape, Permute, BatchNormalization, MaxPooling1D, AveragePooling1D, MaxPooling3D, AveragePooling2D, LayerNormalization, MaxPooling2D
from keras.models import Model
from keras.optimizers import Adam

In [8]:
import os
from os import listdir
from os.path import isfile, join

In [9]:
import pandas as pd
import numpy as np
import copy
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle
import gc

In [10]:
# Set the google cloud bucket data
project_id = 'tweetprediction'
bucket_name = 'crypto_nlp_training'

In [11]:
# Set the checkpoint path for saving train progress
if COLAB:
    CHECKPOINT_PATH = f"gs://{bucket_name}/chk/"
else:
    CHECKPOINT_PATH = f"/content/bigdata/chk/"

In [12]:
# Check if the notebook is run in google colab, if so, clone the repo
if COLAB:
    print("Running in Colab")

    # Clone the whole repo to get all data and code if not already done
    if not os.path.exists("/content/CryptoCrystalBall"):
      !git clone https://github.com/girsigit/CryptoCrystalBall

      # cd into the notebooks directory --> Necessary to match all paths for importing
    #%cd /content/CryptoCrystalBall/JupyterDocker/notebooks
    %cd /content

In [13]:
# Try importing the Ta-Lib library, if this fails, try to install it and
# import it again afterwards
try:
    import talib
except:
    !wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
    !tar -xzvf ta-lib-0.4.0-src.tar.gz
    %cd ta-lib
    !./configure --prefix=/usr
    !make
    !make install
    !pip install Ta-Lib
    %cd ..

    import talib

In [14]:
try:
  from transformers import TFRobertaModel, RobertaConfig
except:
  # Important!: Version 4.23 does not work on TPU
  !pip install transformers==4.22

  from transformers import TFRobertaModel, RobertaConfig

In [15]:
# !pip uninstall -y tensorboard-plugin-profile

In [16]:
# # Required to do profiling
# !pip install tensorboard-plugin-profile

In [17]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

---
# Add custom import path for DataStreamCreator and IndicatorCalculator

These libs are not in the standard python directory, so their paths have to be added to the import paths

In [18]:
# # Get the current directory
# current_dir = os.getcwd()
# current_dir_splitted = current_dir.split(os.sep)

# Todo: is inside /content/CB in local docker
current_dir_splitted = ["", "content"]
current_dir_splitted

['', 'content']

In [19]:
# Create the import directories for the DataStreamCreator and the IndicatorCalculator
dsc_dir = '/content/CryptoCrystalBall/DataStreamCreator'
print(f"dsc_dir: {dsc_dir}")

ind_dir = '/content/CryptoCrystalBall/IndicatorCalculator'
print(f"ind_dir: {ind_dir}")

# Add them to the import paths
sys.path.insert(0, dsc_dir)
sys.path.insert(0, ind_dir)

dsc_dir: /content/CryptoCrystalBall/DataStreamCreator
ind_dir: /content/CryptoCrystalBall/IndicatorCalculator


In [20]:
# Import the actual classes
from IndicatorCalculator import IndicatorCalculator
import DataStreamCreator

---
# Define all the parameters and variables

In [21]:
# Define the tick data path
DATA_PATH = os.path.join(os.sep, *current_dir_splitted, 'DemoData')
print(f"DATA_PATH: {DATA_PATH}")

DATA_PATH: /content/DemoData


In [22]:
# Define the chart image save path
IMG_SAVE_PATH = os.path.join(os.sep, *current_dir_splitted, 'Documentation', 'Images')
print(f"IMG_SAVE_PATH: {IMG_SAVE_PATH}")

IMG_SAVE_PATH: /content/Documentation/Images


In [76]:
# Define a global random seed
RANDOM_SEED = 42+27+1

# Set the seed in np
np.random.seed(RANDOM_SEED)

In [24]:
# X_BLOCK_LENGHT defines how far into the past a 'slice of a chart' shall be
# See: https://github.com/girsigit/CryptoCrystalBall/tree/main/DataStreamCreator#xblockgenerator
X_BLOCK_LENGHT = 64

In [25]:
# How many examples shall be processed at the same time, limited by GPU memory
BATCH_SIZE = 512

In [26]:
# A fixed number of features is used
FEATURES = 160

In [27]:
# Finanical indicator timespans
# See: https://github.com/girsigit/CryptoCrystalBall/tree/main/IndicatorCalculator
SHORTSPAN = 7
MIDSPAN = 38
LONGSPAN = 50

In [28]:
# Additional settings for the data stream
# For this notebook, the calculation of pattern indicators is turned off
DATA_STREAM_PARAMETERS = {
    "calcPatternIndicators": False, # No patterns are used
    "calcVolumeInidators": False, # No volume indicators, these are wide spread and may disturb the classifer
    "dropna": True # Drop all tick/indicator table rows containing nan values instead of just replacing them by 0 (which would lead to wrong predictions)
}

# Load the train dataset

In [29]:
# Check if the dataset already has been copy, if not, copy it
if not os.path.exists("/content/dataset") or not os.path.exists("/content/dataset/Train"):
  !mkdir /content/dataset
  !mkdir /content/dataset/Train
  !gsutil -m cp -r gs://cryptocrystalball_public/CryptoDataset/Hourly/significant_currencies.txt /content/dataset/significant_currencies.txt
  !gsutil -m cp -r gs://cryptocrystalball_public/CryptoDataset/Hourly/Train/* /content/dataset/Train/

In [30]:
#@markdown ### Use only significant currencies
#@markdown Load a manually defined list of significant currencies (`significant_currencies.txt`).
#@markdown This list contains no currencies which little or no volume or price movement, to
#@markdown avoid training on data sample which would never be used to trade on in a real 
#@markdown application.

#@markdown If enabled, only currency pairs with the base currency USDT are laoded,
#@markdown this is important to prevent interference between different cryptocurrencies.
#@markdown For example, in `BTC-ETH.csv`, there is influence of both the BTC and the ETH price, but we want to predict trade signals based on a 'real' currency (USDT is kind of the same as USD).

significant_only = True #@param {type:"boolean"}

if significant_only:
  with open("/content/dataset/significant_currencies.txt") as f:
    SIGNIFICANT_CURRS = f.read().splitlines()

  print(f"Loaded {len(SIGNIFICANT_CURRS)} significant files names.")

Loaded 121 significant files names.


In [31]:
# Get train file names - Only pick the ones ending with -USDT to prevent
# influence between different currencies
TRAIN_PATH = "/content/dataset/Train"

# Get all file names
TRAIN_FILES = [os.path.join(TRAIN_PATH,f) for f in listdir(TRAIN_PATH) if isfile(join(TRAIN_PATH, f)) and ".csv" in f ]

# Filter for significant currencies only
if significant_only:
  TRAIN_FILES = [f for f in TRAIN_FILES if f.split("/")[-1].replace(".csv","") in SIGNIFICANT_CURRS]

# Filter for USDT-based ones only
TRAIN_FILES = [f for f in TRAIN_FILES if "-USDT" in f]

# Sort them (as a stable basis for randomizing afterwards)
TRAIN_FILES = sorted(TRAIN_FILES)

print(f"The train dataset contains {len(TRAIN_FILES)} files.")
print(TRAIN_FILES[:3])

The train dataset contains 121 files.
['/content/dataset/Train/1INCH-USDT.csv', '/content/dataset/Train/4ART-USDT.csv', '/content/dataset/Train/AAVE-USDT.csv']


---
# Prepare data source

For training a neural network, first the data source has to be prepared. For this purpose, the method `FileListToDataStream` from the `DataStreamCreator` class is used. This method creates a stream of `X-Block` and `y-data` arrays out of a list of .csv file names, pointing to tick tables (called `EXAMPLE_FILE_PATHS` in this example). For details about `X-Blocks` and `y-data`, please refer to the documentation of the `XBlockGenerator` and the `YDataGenerator` under https://github.com/girsigit/CryptoCrystalBall/tree/main/DataStreamCreator.

<br>

Target values (y-data) from the data generator would not be necessary in this notebook, but since it cannot be switched off, the future direction and its derviation of the price have been chosen in `Y_TYPE_DICT` since they are not expensive to compute. A switch flag will be added in a future release.

In [32]:
# Set direction and derivation information as y target
# Both y values (direction & derivation) are in the interval [-1.0,1.0]

Y_TYPE_DICT = copy.deepcopy(DataStreamCreator.YDataGenerator.PARAM_DICT_TEMPLATE_Y_DATA_TYPE_DIRECTION_FLOAT)
Y_TYPE_DICT

{'dataType': 0,
 'direction_ma_timespan': 200,
 'derivation_ma_timespan': 100,
 'direction_derivation_shift_span': 0}

In [33]:
TARGET_FEATURE_CNT = 256

In [34]:
XShape = (BATCH_SIZE, X_BLOCK_LENGHT, TARGET_FEATURE_CNT)
XShape

(512, 64, 256)

In [77]:
# A python generator function has to be applied on the dataStream

def pythonGeneratorMLMTraining():
  # Initialize the FileListToDataStream generator
  dataStreamTraining = DataStreamCreator.FileListToDataStream(fileList = TRAIN_FILES,
                                                      batch_size = BATCH_SIZE,
                                                      X_Block_lenght = X_BLOCK_LENGHT,
                                                      y_type_dict=Y_TYPE_DICT,
                                                      shuffle=True,
                                                      parallel_generators = np.min([BATCH_SIZE, 8]),
                                                      random_seed = RANDOM_SEED,
                                                      **DATA_STREAM_PARAMETERS
                                                      )

  # This while has to integrated into the FileListToDataStream method
  while True:  
    try:
      ne = next(dataStreamTraining)
      _X = ne['X']
      _y = ne['y']
    
      # 'Blow up' the X array to a 2**n size
      _X_new = np.zeros((_X.shape[0], _X.shape[1], TARGET_FEATURE_CNT))
      _X_new[:,:,:_X.shape[2]] = _X

      yield (_X_new, _y)
    except StopIteration as si:
      logging.warning("StopIteration in pythonGenerator")
      logging.warning(si)
      return

In [78]:
# Create a Tensorflow dataset out of the python generator, which can be fed to the network
tfGenTraining = tf.data.Dataset.from_generator(pythonGeneratorMLMTraining, 
                                               output_types = (tf.int32, tf.float32),
                                               output_shapes=(
                                                   (BATCH_SIZE, X_BLOCK_LENGHT, TARGET_FEATURE_CNT),
                                                   (BATCH_SIZE, 2)
                                                   )
                                               )
tfGenTraining.prefetch(buffer_size=2)
tfGenTraining

<FlatMapDataset element_spec=(TensorSpec(shape=(512, 64, 256), dtype=tf.int32, name=None), TensorSpec(shape=(512, 2), dtype=tf.float32, name=None))>

In [37]:
# it = tfGenTraining.as_numpy_iterator()
# ne = next(it)

In [38]:
# ne[0]

In [39]:
# ne[1]

In [40]:
# plt.hist(ne[0].flatten())

In [41]:
# plt.hist(ne[1].flatten())

In [42]:
# wurst = np.array(ne[1])
# wurst[ne[0] != MASK_TOKEN_ID] = -1
# wurst

In [43]:
# np.sum(ne[0][0,:] == 3)

---
# Create the neural network

In the example, two Conv1D pyramids are used to extract features at different levels out of the feature and the time dimension.

It is also possible to use different architecures, like LSTM, Conv2D, Attention-Based ones or even NLP-based models. I have already tried many different ones and will publish them into the model zoo.

```
Todo: Create Model Zoo
```

```
Todo: Create Image of netwrok structure
```

In [44]:
# from keras.layers import Input, Dense, ReLU, Add, Flatten, Concatenate, LayerNormalization, UpSampling2D, Activation, LSTM, Multiply, Dropout, Reshape, Permute, BatchNormalization, MaxPooling1D, AveragePooling1D, MaxPooling3D, AveragePooling2D, LayerNormalization, MaxPooling2D, UnitNormalization, UpSampling1D
from keras.layers import UnitNormalization, Permute, Conv1D, UpSampling1D, Add, LSTM


In [45]:
# Create the FPNWithFScaledInput model
# Infos about the feature pyramid: https://jonathan-hui.medium.com/understanding-feature-pyramid-networks-for-object-detection-fpn-45b227b9106c

def CreateModelFPNWithFScaledInput():
  # Define model parameters
  mp = {
      "NAME": "FPNWithFScaledInput",
      "VERSION": 3,
      "CNN_INITIAL_FILTERS": 32, 
      "CNN_EXTRACTOR_LAYERS": 6,
      "CNN_INITIAL_KERNEL_SIZE": 2,
      "CNN_MAX_KERNEL_SIZE": 8,
      "FEATURE_PYRAMID_START_INDEX": 2,
      "HEAD_LSTM_SIZE": 32,
      "HEAD_DENSE_SIZE": 8
      }
  
  # Create the model input  
  inputTicksAndIndicators = Input(shape=(X_BLOCK_LENGHT, TARGET_FEATURE_CNT), name='inputTicksAndIndicators', dtype='float32')

  # Normalize the input data, as it has a wide value spread (range of 1e8)
  normalized = UnitNormalization(name="UnitNormalizationInput", axis=-2)(inputTicksAndIndicators)

  # Permute it to Conv over Features
  normalized = Permute((2, 1), name="PermuteInput")(normalized)

  # ----------------------------------------------------------------------------

  # As an example a 1D-CNN pyramid is used to extract higher-level features out
  # of the indicator+tick dimension
  conv_layers = []
  aligned_layers = []
  feature_map_layers = []
  anti_aliased_layers = []
  anti_aliased_pyramid_indices = []
  regressor_layers = []

  finalFilters = mp["CNN_INITIAL_FILTERS"] * 2**(mp["FEATURE_PYRAMID_START_INDEX"])
  print(finalFilters)

  for i in range(mp["CNN_EXTRACTOR_LAYERS"]):
    if 0 == i:
      source_layer = normalized
    else:
      source_layer = feature_conv
    
    filters = int(mp["CNN_INITIAL_FILTERS"]*(2**i))
    kernel_size = mp["CNN_INITIAL_KERNEL_SIZE"]*(2**i)
    kernel_size = int(np.min([mp["CNN_MAX_KERNEL_SIZE"], kernel_size]))
    strides = int(np.min([2,kernel_size]))

    print(f"Step {i}, using {filters} filters with a kernel size of {kernel_size} at strides of {strides}")

    feature_conv = Conv1D(filters=filters,
                          kernel_size=kernel_size,
                          strides=strides, 
                          padding="same",
                          name=f"Feature_Conv1D_{i}")(source_layer)
    
    conv_layers.append(feature_conv)

    # Apply the feature extraction pyramid
    if mp["FEATURE_PYRAMID_START_INDEX"] <= i:
      # Apply a 1x1 convolution to align the channel depth    
      if feature_conv.shape[-1] != finalFilters:
        aligned_layer = Conv1D(filters=finalFilters,
                              kernel_size=1,
                              strides=1,
                              padding="same",
                              name=f"Channel_Depth_Alignment_{i}")(feature_conv)
      else:
        aligned_layer = feature_conv
      
      aligned_layers.append(aligned_layer)

  # Go top-down through the aligned_layers to create the feature_map_layers
  for i in range(mp["CNN_EXTRACTOR_LAYERS"]-1, mp["FEATURE_PYRAMID_START_INDEX"]-1, -1):
    aligned_layers_index = i - mp["FEATURE_PYRAMID_START_INDEX"]
    aligned_layer = aligned_layers[aligned_layers_index]

    print(f"Building feature_map_layers, step {i}. Shape of aligned_layer: {aligned_layer.shape}")

    # The highest-filtered layer is taken as feature map directly
    if i == mp["CNN_EXTRACTOR_LAYERS"]-1:
      feature_map_layers.append(aligned_layer)
      anti_aliased_layers.append(feature_map_layers[-1])
    else:
      # Take the last feature_map_layer and scale it by two
      upsampled = UpSampling1D(size=2,
                               name=f"Pyramid_Upsampling_{i}")(feature_map_layers[-1])

      # Add the aligned_layer
      added = Add(name=f"Pyramid_Add_{i}")([upsampled, aligned_layer])

      # Append as new feature_map_layer
      feature_map_layers.append(added)

      # Apply a convolution with a kernel size of 3 to "reduce the aliasing effect"
      anti_aliased_layer = Conv1D(filters=feature_map_layers[-1].shape[-1],
                                  kernel_size=3,
                                  strides=1,
                                  padding="same",
                                  name=f"Anti_Alias_{i}")(feature_map_layers[-1])

      anti_aliased_layers.append(anti_aliased_layer)

    # Helper storage to keep pyramid index consistent, also in predictor/regressor
    anti_aliased_pyramid_indices.append(i)

  # Apply the predictor head to each feature dimension layer
  for n in range(len(anti_aliased_layers)):
    predictor_input = anti_aliased_layers[n]

    # Get the pyramid index
    pyramid_index = anti_aliased_pyramid_indices[n]

    # Permute the Conv output back
    predictor_input = Permute((2, 1), name=f"PermutePredictorInput_{pyramid_index}")(predictor_input)

    predictor = LSTM(units=mp["HEAD_LSTM_SIZE"],
                     name=f"Predictor_Head_{pyramid_index}_LSTM")(predictor_input)
    
    # Direction and derivation regressor
    regressor = Dense(units=mp["HEAD_DENSE_SIZE"],
                      name=f"Predictor_Head_{pyramid_index}_Dense_Regressor",
                      activation='relu')(predictor)
    
    regressor_layers.append(regressor)
  
  # Add all regressors together
  regressor_added = Add(name="Add_regressors")(regressor_layers)

  # The output are two values (direction and derivation) in the range [-1.0, 1.0]
  output = Dense(2, activation='tanh', name="Output")(regressor_added)
  outputs = [output]

  # Combine it all into a model object
  model = Model(inputs=inputTicksAndIndicators, outputs=outputs, name=mp["NAME"] + "_" + str(mp["VERSION"]))

  return model, mp

model, model_config = CreateModelFPNWithFScaledInput()
print(model_config)
model.summary(line_length=220)

128
Step 0, using 32 filters with a kernel size of 2 at strides of 2
Step 1, using 64 filters with a kernel size of 4 at strides of 2
Step 2, using 128 filters with a kernel size of 8 at strides of 2
Step 3, using 256 filters with a kernel size of 8 at strides of 2
Step 4, using 512 filters with a kernel size of 8 at strides of 2
Step 5, using 1024 filters with a kernel size of 8 at strides of 2
Building feature_map_layers, step 5. Shape of aligned_layer: (None, 4, 128)
Building feature_map_layers, step 4. Shape of aligned_layer: (None, 8, 128)
Building feature_map_layers, step 3. Shape of aligned_layer: (None, 16, 128)
Building feature_map_layers, step 2. Shape of aligned_layer: (None, 32, 128)
{'NAME': 'FPNWithFScaledInput', 'VERSION': 3, 'CNN_INITIAL_FILTERS': 32, 'CNN_EXTRACTOR_LAYERS': 6, 'CNN_INITIAL_KERNEL_SIZE': 2, 'CNN_MAX_KERNEL_SIZE': 8, 'FEATURE_PYRAMID_START_INDEX': 2, 'HEAD_LSTM_SIZE': 32, 'HEAD_DENSE_SIZE': 8}
Model: "FPNWithFScaledInput_3"
______________________________

In [46]:
# modelMLM.load_weights("/content/bigdata/chk/RobertaMLMTraining_1_GPU_512LB_10000VC_MaskedPrediction/cp_daily_valid_03_144000/model.h5")

In [47]:
# model.get_layer("CryptoRoberta").set_weights(modelMLM.get_layer("tf_roberta_model").get_weights())

In [48]:
# model.get_layer("CryptoRoberta").trainable=False

In [49]:
# model.summary()

In [50]:
# model.save("/content/bigdata/RobertaToDirDerv_1_init.h5")

In [51]:
# model.load_weights("/content/bigdata/RobertaToDirDerv_1_init.h5")
#model.load_weights("/content/bigdata/chk/RobertaToDirDerv_1_GPU_512LB_10000VC_MaskedPrediction/cp_daily_valid_00_18000/model.h5")

# Train the model

### Train strategy in the paper
https://huggingface.co/roberta-base

The model was trained on 1024 V100 GPUs for 500K steps with a batch size of 8K and a sequence length of 512. The optimizer used is Adam with a learning rate of 6e-4, β1=0.9\beta_{1} = 0.9β1​=0.9, β2=0.98\beta_{2} = 0.98β2​=0.98 and ϵ=1e−6\epsilon = 1e-6ϵ=1e−6, a weight decay of 0.01, learning rate warmup for 24,000 steps and linear decay of the learning rate after.

In [52]:
CHKPNT_NAME = f"{model.name}_GPU_{X_BLOCK_LENGHT}LB"
CHKPNT_NAME

'FPNWithFScaledInput_3_GPU_64LB'

In [53]:
# Set an optimizer
optimizer = Adam(
    learning_rate=5e-04,
    epsilon=1e-06,
    beta_1=0.9,
    beta_2=0.98,
    decay=0.01)

In [54]:
# Compile the model
model.compile(
    optimizer = optimizer,
    loss = ["mse"], 
    metrics=["mae"])

In [55]:
# model.summary(line_length=220)

In [56]:
#@title CustomCallback
class CustomCallback(tf.keras.callbacks.Callback):
  def __init__(self, save_freq, val_freq, checkpoint_path, model_name, epoch_add=0):
    self.save_freq = save_freq
    self.val_freq = val_freq
    self.checkpoint_path = checkpoint_path
    self.model_name = model_name
    self.current_epoch = 0
    self.epoch_add = epoch_add

  def on_epoch_begin(self, epoch, logs=None):
    self.current_epoch = epoch + self.epoch_add
    # keys = list(logs.keys())
    # print("Start epoch {} of training; got log keys: {}".format(epoch, keys))

  def on_epoch_end(self, epoch, logs=None):
    self.saveTheModel(-1, logs)

  def on_train_batch_end(self, batch, logs=None):
    self.saveTheModel(batch, logs)

  def saveTheModel(self, batch, logs=None):
    if (0 < batch and 0 == batch % self.save_freq) or (0 > batch):
      logging.info(str(datetime.datetime.utcnow()))
    
      if 0 > batch:
        _save_folder = os.path.join(self.checkpoint_path,
                                    self.model_name,
                                    "cp_daily_valid_{:02d}_end".format(self.current_epoch)
                                    )
      else:
        _save_folder = os.path.join(self.checkpoint_path,
                                    self.model_name,
                                    "cp_daily_valid_{:02d}_{:05d}".format(self.current_epoch, batch)
                                    )
      
      fp = os.path.join(_save_folder, "model.h5")
      model.save(fp)
      logging.info(f"Saved model to '{fp}'")
      
      # Save optimizer config
      # c = copy.deepcopy(self.model.optimizer.get_config())

      fp = os.path.join(_save_folder, "c.pickle")
      with file_io.FileIO(fp, mode='wb+') as handle:
        pickle.dump(self.model.optimizer.get_config(), handle, protocol=pickle.HIGHEST_PROTOCOL)
        
      logging.info(f"Saved optimizer config to '{fp}'")

      # Save optimizer weights
      # w = copy.deepcopy(self.model.optimizer.get_weights())

      fp = os.path.join(_save_folder, "w.pickle")
      with open(fp, "wb") as handle:
        # with file_io.FileIO(fp, mode='wb+') as handle:
        pickle.dump(self.model.optimizer.get_weights(), handle, protocol=pickle.HIGHEST_PROTOCOL)
      
      logging.info(f"Saved optimizer weights to '{fp}'")
        
      logging.info(f"Did a gc collect: {gc.collect()}")

    # if 0 < batch and 0 == batch % self.val_freq:
    #   print("-------------------------EVAL-------------------------")
    #   model.evaluate(tfgenTest)
    #   print("\n-------------------------EVAL-------------------------")

In [57]:
gc.collect()

6546

In [58]:
# CLASS_WEIGHTS_DICT = None
# CLASS_WEIGHTS_DICT = {}

# for i, cw in enumerate(CLASS_WEIGHTS[0,0,:]):
#   CLASS_WEIGHTS_DICT[i] = cw

In [72]:
epoch_add = 3

In [73]:
CALLBACK_EVERY_N_BATCHES = 2000

In [74]:
cc = CustomCallback(checkpoint_path = CHECKPOINT_PATH,
                    model_name = CHKPNT_NAME,
                    save_freq = CALLBACK_EVERY_N_BATCHES,
                    val_freq = CALLBACK_EVERY_N_BATCHES,
                    epoch_add = epoch_add)

In [62]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [63]:
%reload_ext tensorboard

The TensorBoard UI is displayed in a browser window. In this colab, perform the following steps to prepare to capture profile information.
1.  Click on the dropdown menu box on the top right side and scroll down and click PROFILE. A new window appears that shows: **No profile data was found** at the top.
1.  Click on the CAPTURE PROFILE button. A new dialog appears. The top input line shows: **Profile Service URL or TPU name**. Copy and paste the Profile Service URL (the service_addr value shown before launching TensorBoard) into the top input line. While still on the dialog box, start the training with the next step.
1.  Click on the next colab cell to start training the model.
1.  Watch the output from the training until several epochs have completed. This allows time for the profile data to start being collected. Return to the dialog box and click on the CAPTURE button. If the capture succeeds, the page will auto refresh and redirect you to the profiling results.

In [64]:
# Todo: Create more elegant solution
log_dir = "gs://ticks_with_indicators_with_volume/logs/TPU/" + CHKPNT_NAME + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

if not COLAB:
    log_dir = os.path.join("/content/bigdata/log",log_dir.split("/")[-1])

log_dir

'/content/bigdata/log/FPNWithFScaledInput_3_GPU_64LB20230227-152030'

In [65]:
log_dir

'/content/bigdata/log/FPNWithFScaledInput_3_GPU_64LB20230227-152030'

In [66]:
import datetime

tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,
    update_freq=CALLBACK_EVERY_N_BATCHES,
    profile_batch=(5,100)
    )

2023-02-27 15:20:30.549074: I tensorflow/core/profiler/lib/profiler_session.cc:101] Profiler session initializing.
2023-02-27 15:20:30.549096: I tensorflow/core/profiler/lib/profiler_session.cc:116] Profiler session started.
2023-02-27 15:20:30.550850: I tensorflow/core/profiler/backends/gpu/cupti_tracer.cc:1664] Profiler found 1 GPUs
2023-02-27 15:20:30.551178: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcupti.so.11.2'; dlerror: libcupti.so.11.2: cannot open shared object file: No such file or directory
2023-02-27 15:20:30.673719: I tensorflow/core/profiler/lib/profiler_session.cc:128] Profiler session tear down.
2023-02-27 15:20:30.673860: I tensorflow/core/profiler/backends/gpu/cupti_tracer.cc:1798] CUPTI activity buffer flushed


In [67]:
# CLASS_WEIGHTS_DICT = None

In [68]:
# model.load_weights("/content/bigdata/chk/RobertaMLMTraining_1_GPU_512LB_10000VC_MaskedPrediction/cp_daily_valid_01_end/model.h5")

In [75]:
# model.optimizer.learning_rate = 5e-4

In [70]:
%tensorboard --logdir $log_dir --host 0.0.0.0 --port 6006

In [ ]:
model.fit(tfGenTraining,
          epochs=200,
          verbose = 1,
          callbacks=[tensorboard_callback, cc])

Epoch 1/200


INFO:root:File 'OMG-USDT.csv' loaded, 120 left
INFO:root:File 'REV-USDT.csv' loaded, 119 left
INFO:root:File 'XLM-USDT.csv' loaded, 118 left
INFO:root:File 'CRFI-USDT.csv' loaded, 117 left
INFO:root:File 'KLAY-USDT.csv' loaded, 116 left
INFO:root:File 'KDA-USDT.csv' loaded, 115 left
INFO:root:File '1INCH-USDT.csv' loaded, 114 left
INFO:root:File 'HIVE-USDT.csv' loaded, 113 left


      4/Unknown - 6s 253ms/step - loss: 0.0753 - mae: 0.1609

2023-02-27 15:57:41.303312: I tensorflow/core/profiler/lib/profiler_session.cc:101] Profiler session initializing.
2023-02-27 15:57:41.303338: I tensorflow/core/profiler/lib/profiler_session.cc:116] Profiler session started.


     10/Unknown - 7s 254ms/step - loss: 0.0980 - mae: 0.1851

...

INFO:root:File 'CLT-USDT.csv' loaded, 95 left


 376/2343 [===>..........................] - ETA: 8:36 - loss: 0.0938 - mae: 0.1830

INFO:root:File 'GAME-USDT.csv' loaded, 94 left


 408/2343 [====>.........................] - ETA: 8:26 - loss: 0.0935 - mae: 0.1827

INFO:root:File 'XRP-USDT.csv' loaded, 93 left


In [ ]:
# copy_filenames = ['gs://crypto_nlp_training/chk/RobertaMLMTraining_1_GPU_512LB_10000VC_MaskedPrediction/cp_daily_valid_01_06000/model.h5'
#                   ]

# for p in copy_filenames:
#   fn = p.split("/")[-1]
#   cpnt = p.split("/")[-2]

#   os.mkdir(os.path.join("/content", cpnt))

#   localPath = os.path.join("/content", cpnt, fn)

#   if ("model.h5" in p):
#     localPathModel = localPath
#   elif ("w.pickle" in p):
#     localPathW = localPath

#   with file_io.FileIO(p, mode='rb') as input_f:
#     with file_io.FileIO(localPath, mode='wb+') as output_f:
#       output_f.write(input_f.read())
#       print("Pulled from bucket: '" + fn + "'")

In [ ]:
# print(f"Loading {localPathModel}")
# model.load_weights(localPathModel)

In [ ]:
# model.load_weights("/content/model.h5")

In [ ]:
# modelOld = CreateModelRobertaMLMTrainingDropout()
# modelOld.load_weights("/content/cp_daily_valid_49_end/model.h5")
# modelOld.summary()

In [ ]:
# model.get_layer("Roberta").set_weights(modelOld.get_layer("Roberta").get_weights())
# model.get_layer("Categories").set_weights(modelOld.get_layer("Categories").get_weights())

In [ ]:
# with open("/content/w.pickle", 'rb') as pickle_file:
#   w = pickle.load(pickle_file)
# model.optimizer.set_weights(w)

In [ ]:
# model.optimizer.learning_rate = 5e-2

LR 5e-4  5000/Unknown - 5576s 1s/step - loss: 296.81093730


# Full sentence as y
20000/Unknown - 21155s 1s/step - loss: 0.9579
Saved model to: 'gs://crypto_nlp_training/chk/RobertaMLMTraining_1_GPU_512LB_10000VC_FullPrediction/cp_daily_valid_00_20000/model.h5'

# Masked sentence as y, LR 5e-4, Random state 11
14000/Unknown - 15387s 1s/step - loss: 5.3956
Saved model to: 'gs://crypto_nlp_training/chk/RobertaMLMTraining_1_GPU_512LB_10000VC_MaskedPrediction/cp_daily_valid_00_14000/model.h5'

In [ ]:
# A python generator function has to be applied on the dataStream

def pythonGeneratorMLMTraining():
  # Initialize the FileListToDataStream generator
  dataStreamTraining = DataStreamCreator.FileListToDataStream(fileList = TRAIN_FILES,
                                                      batch_size = BATCH_SIZE,
                                                      X_Block_lenght = X_BLOCK_LENGHT,
                                                      y_type_dict=Y_TYPE_DICT,
                                                      shuffle=True,
                                                      parallel_generators = 8,
                                                      random_seed = RANDOM_SEED,
                                                      **DATA_STREAM_PARAMETERS
                                                      )
  
  # Calculate how many word shall be replaced
  mask_number = int(np.round(X_BLOCK_LENGHT * MLM_MASK_FACTOR))
  logging.info(f"In each batch of X-Blocks, {mask_number*BATCH_SIZE} elements will be randomly masked. This is an average of {mask_number} per X-Block")

  # This while has to integrated into the FileListToDataStream method
  while True:  
    try:
      ne = next(dataStreamTraining)
      _X = ne['X']

      # Convert the X-Block into a sentence
      with tf.device('/CPU:0'):        
        four_float_sentence = fourFloatClassifierModel.predict(_X, batch_size = BATCH_SIZE, verbose = 0)
        int_sentence = ConvertFourFloatDataToWords(four_float_sentence, digitLimits)

        # _X_sentence = intClassifierModel.predict(_X, batch_size = BATCH_SIZE, verbose = 0)

      # Round to avoid too many categories
      # Todo: Better classifier model!
      # _X_sentence = np.round(_X_sentence / 10.0).astype(int)

      # Create random indices, to replace 'words' with MASK_TOKEN_I
      mask_positions = np.round(np.random.rand(BATCH_SIZE, mask_number) * X_BLOCK_LENGHT * BATCH_SIZE).astype(int)
      mask_positions[mask_positions == X_BLOCK_LENGHT * BATCH_SIZE] -= 1  # Avoid the upper array limit

      # Mask the chosen tokens
      int_sentence_masked = np.array(int_sentence).flatten()
      int_sentence_masked[mask_positions] = MASK_TOKEN_ID
      int_sentence_masked = int_sentence_masked.reshape(int_sentence.shape)

      # print(mask_positions[0,0])

      # 'Remove' all tokens that shall not be predicted from the training y data (the full sentence), so that the network can focus on the missing tokens
      # More precise: Setting them to -1 tells the loss function to ignore them
      int_sentence[int_sentence_masked != MASK_TOKEN_ID] = -1

      # Not required here, as the network shall predict back its original input
      # _y = ne['y']
      
      # Return the masked senteces as X data, the full ones are the y-data --> The network shall predict the missing tokens
      yield (int_sentence_masked, int_sentence)
    except StopIteration as si:
      logging.warning("StopIteration in pythonGenerator")
      logging.warning(si)
      return

In [ ]:
# A python generator function has to be applied on the dataStream

def pythonGeneratorMLMEval():
  # Initialize the FileListToDataStream generator
  dataStreamTraining = DataStreamCreator.FileListToDataStream(fileList = TRAIN_FILES,
                                                      batch_size = BATCH_SIZE,
                                                      X_Block_lenght = X_BLOCK_LENGHT,
                                                      y_type_dict=Y_TYPE_DICT,
                                                      shuffle=True,
                                                      parallel_generators = BATCH_SIZE,
                                                      random_seed = RANDOM_SEED,
                                                      **DATA_STREAM_PARAMETERS
                                                      )
  
  # Calculate how many word shall be replaced
  mask_number = int(np.round(X_BLOCK_LENGHT * MLM_MASK_FACTOR))
  logging.info(f"In each batch of X-Blocks, {mask_number*BATCH_SIZE} elements will be randomly masked. This is an average of {mask_number} per X-Block")

  # This while has to integrated into the FileListToDataStream method
  while True:  
    try:
      ne = next(dataStreamTraining)
      _X = ne['X']

      # Convert the X-Block into a sentence
      with tf.device('/CPU:0'):        
        four_float_sentence = fourFloatClassifierModel.predict(_X, batch_size = BATCH_SIZE, verbose = 0)
        int_sentence = ConvertFourFloatDataToWords(four_float_sentence, digitLimits)

        # _X_sentence = intClassifierModel.predict(_X, batch_size = BATCH_SIZE, verbose = 0)

      # Round to avoid too many categories
      # Todo: Better classifier model!
      # _X_sentence = np.round(_X_sentence / 10.0).astype(int)

      # Create random indices, to replace 'words' with MASK_TOKEN_I
      mask_positions = np.round(np.random.rand(BATCH_SIZE, mask_number) * X_BLOCK_LENGHT * BATCH_SIZE).astype(int)
      mask_positions[mask_positions == X_BLOCK_LENGHT * BATCH_SIZE] -= 1  # Avoid the upper array limit

      # Mask the chosen tokens
      int_sentence_masked = np.array(int_sentence).flatten()
      int_sentence_masked[mask_positions] = MASK_TOKEN_ID
      int_sentence_masked = int_sentence_masked.reshape(int_sentence.shape)

      # print(mask_positions[0,0])

      # 'Remove' all tokens that shall not be predicted from the training y data (the full sentence), so that the network can focus on the missing tokens
      # More precise: Setting them to -1 tells the loss function to ignore them
      # int_sentence[int_sentence_masked != MASK_TOKEN_ID] = -1

      # Not required here, as the network shall predict back its original input
      # _y = ne['y']
      
      # Return the masked senteces as X data, the full ones are the y-data --> The network shall predict the missing tokens
      yield (int_sentence_masked, int_sentence)
    except StopIteration as si:
      logging.warning("StopIteration in pythonGenerator")
      logging.warning(si)
      return

In [ ]:
# Create a Tensorflow dataset out of the python generator, which can be fed to the network
tfGenEval = tf.data.Dataset.from_generator(pythonGeneratorMLMEval, 
                                               output_types = (tf.int32, tf.int32),
                                               output_shapes=(
                                                   (BATCH_SIZE, X_BLOCK_LENGHT),
                                                   (BATCH_SIZE, X_BLOCK_LENGHT)
                                                   )
                                               )
tfGenEval

In [ ]:
it = tfGenEval.as_numpy_iterator()
ne = next(it)

In [ ]:
p = model.predict(ne[0])
p.shape

In [ ]:
gtVal = ne[1][0,:]
gtVal.shape

In [ ]:
porig= copy.deepcopy(p)
# p = copy.deepcopy(porig)

In [ ]:
plt.plot(p[0,0,:])

In [ ]:
intPrediction = np.empty(p.shape[:-1])

for b in range(p.shape[0]):
  for ts in range(p.shape[1]):
    classMax = np.max(p[b,ts,:])
    maxIndex = np.where(p[b,ts,:] == classMax)

    #print(f"classMax: {classMax}, maxIndex: {maxIndex}")
    
    intPrediction[b,ts] = maxIndex[0]

In [ ]:
intPrediction.shape

In [ ]:
gtVal[:20]

In [ ]:
intPrediction[0,:50]

In [ ]:
_ = plt.hist(gtVal.flatten(), bins=50)

In [ ]:
_ = plt.hist(intPrediction.flatten(), bins=50)

In [ ]:
plt.plot(intPrediction[0,:])

In [ ]:
pMaxed = p * (p >= np.sort(p, axis=2)[:,[-1]])

In [ ]:
pMaxes = np.max(p, axis=2)
pMaxes.shape

In [ ]:
# p == pMaxes

In [ ]:
# p[:] = np.where(p == pMaxes, pMaxes, 0)

In [ ]:
plt.plot(p[0,0,:])

In [ ]:
np.max(p[0,0,:])

In [ ]:
pMaxes[0,0]

In [ ]:
plt.plot(pMaxed[0,0,:])

In [ ]:
# _save_folder = os.path.join(checkpoint_path,
#                                 CHKPNT_NAME,
#                                 "robert_tpu_scratch_{:01d}_{:05d}".format(112, 0)
#                                 )

# _model_path_local = os.path.join("/content/", "model.h5")
# _model_path_bucket = os.path.join(_save_folder, "model.h5")

# model.save(_model_path_local)

# # Copy model.h5 over to Google Cloud Storage
# with file_io.FileIO(_model_path_local, mode='rb') as input_f:
#     with file_io.FileIO(_model_path_bucket, mode='wb+') as output_f:
#         output_f.write(input_f.read())
#         print("\nSaved model to: '" + _model_path_bucket + "'")

# # Save optimizer config
# c = copy.deepcopy(model.optimizer.get_config())

# fp = os.path.join(_save_folder, "c.pickle")
# with file_io.FileIO(fp, mode='wb+') as handle:
#   pickle.dump(c, handle, protocol=pickle.HIGHEST_PROTOCOL)
#   print("Saved optimizer config to: '" + fp + "'")

# # Save optimizer weights
# w = copy.deepcopy(model.optimizer.get_weights())

# fp = os.path.join(_save_folder, "w.pickle")
# with file_io.FileIO(fp, mode='wb+') as handle:
#   pickle.dump(w, handle, protocol=pickle.HIGHEST_PROTOCOL)
#   print("Saved optimizer weights to: '" + fp + "'")